<a href="https://colab.research.google.com/github/Eisenh/AMPP2024_s-CO2_Corr_Pred/blob/main/applications/biomass-gasification/biomass-gasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Biomass Gasification

<p class="acknowledgement">Written by Prof. William Smith (University of Guelph) and Allan Leal (ETH Zurich) on Mar 31st, 2023</p>

This tutorial demonstrates the use of Reaktoro for biomass gasification in which temperature and exit gas composition are calculated from given Fuel Ultimate Analysis and fuel-to-air ratios.

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:33
🔁 Restarting kernel...
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - reaktoro


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5        

In [1]:
from reaktoro import *
import reaktoro as rk
from numpy import arange

In [2]:
from pprint import pprint
pprint(vars(rk.Reaction))

mappingproxy({'__doc__': None,
              '__init__': <instancemethod __init__ at 0x7ca12b02b520>,
              '__module__': 'reaktoro.reaktoro4py',
              '_pybind11_conduit_v1_': <instancemethod _pybind11_conduit_v1_ at 0x7ca12b02b4c0>,
              'clone': <instancemethod clone at 0x7ca12b02b580>,
              'equation': <instancemethod equation at 0x7ca12b02b760>,
              'name': <instancemethod name at 0x7ca12b02b700>,
              'props': <instancemethod props at 0x7ca12b02b850>,
              'rate': <instancemethod rate at 0x7ca12b02b880>,
              'rateModel': <instancemethod rateModel at 0x7ca12b02b7c0>,
              'withEquation': <instancemethod withEquation at 0x7ca12b02b640>,
              'withName': <instancemethod withName at 0x7ca12b02b5e0>,
              'withRateModel': <instancemethod withRateModel at 0x7ca12b02b6a0>})


In [4]:
#db = NasaDatabase("nasa-cea")
db = rk.SupcrtDatabase("supcrt16")
db.extendWithDatabase(SupcrtDatabase("supcrt16-organics"))

In [5]:
species_list = db.species()
species_list_size = db.species().size()
print(species_list_size)

3097


In [10]:
# SEARCH
searchterm = "Lime"
for spec in species_list:
    if searchterm in spec.name():
        print("Found ", spec.name() )
        break
else:
    print(searchterm, " not found in the database.")

Found  Lime


In [7]:
#for spec in species_list:
#    print(spec.name(), spec.formula())

solution = AqueousPhase("H2O(aq) H+ OH- Na+ NaOH(aq) CO3-2 CO(aq) CO(g) CaOH+ Calcite Lime (-COOH)(aq) CO2(aq) CO3-2 Ca(HCO3)+ Ca+2 CaCO3(aq) FeO(aq) CaOH+")
# Because we want to compute the solubility of halite, a mineral phase is needed
solids = MineralPhase("Calcite Lime")

HTLsystem = ChemicalSystem(db, solution, solids)
for phase in HTLsystem.phases():
    print(phase.name())
    for species in phase.species():
        print(":: " + species.name())

# all_reactions_size = all_reactions.size()
# Print the species names

#for rx in spec.:
#  print(spec.reaction().stoichiometry())


RuntimeError: [1;31m***ERROR*** Could not find any Species object with name CaO.
[0m

In [ ]:
# Inputs -----------------------------------------------------------------------------
massC              = 49.30  # g/mol
massH              = 5.5    # g/mol
massO              = 45.2   # g/mol
HHV                = 18.933 # kJ/g
Q                  = 0.05   # heat loss (%) with respect to HHV
fAirMin            = 0.70   # minimum value for fAir (fuel to air mass ratio)
fAirDelta          = 0.1    # step-size for fAir values
fAirNum            = 30     # number of steps for fAir
output_all_species = True   # if false, output only a few species
# ------------------------------------------------------------------------------------

In [ ]:
fAirs = [fAirMin + i * fAirDelta for i in range(fAirNum)]

nC = massC / 12.011
nH = massH / 1.00797
nO = massO / 15.9994

h0CO2 = -393.522  # kJ/g
h0H2O = -285.83   # kJ/g

a = nH / nC
b = nO / nC

formula = ChemicalFormula(f"CH{a}O{b}")

Mfuel = formula.molarMass() * 1000  # from kg/mol to g/mol
Mair = 28.850334  # in g/mol

factor = Mfuel / Mair

HHVmol = HHV * Mfuel

In [ ]:
h0fuel = HHVmol + h0CO2 + 0.5*h0H2O*a  # in J/mol
heatDuty = (1 - Q) * HHVmol + h0CO2 + 0.5*h0H2O*a  # in J/mol

In [ ]:
stmodelparams = rk.StandardThermoModelParamsConstant()
stmodelparams.G0 = 1.0e+3
stmodel = rk.StandardThermoModelConstant(stmodelparams)

species = rk.Species()

In [ ]:

# pprint(species)
print(species.standardThermoModel.__dir__())

['__new__', '__repr__', '__hash__', '__call__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__get__', '__reduce__', '__func__', '__self__', '__doc__', '__str__', '__init__', '__reduce_ex__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


In [ ]:

species = species.withName("Fuel")
species = species.withElements(formula.elements())
species = species.withAggregateState(AggregateState.CondensedPhase)
species = species.withStandardThermoModel(stmodel)

db.addSpecies(species)

In [ ]:
gases = GaseousPhase(speciate("C H O N"))
condensedphases = CondensedPhases("Fuel H2O(l) C(gr)")

system = ChemicalSystem(db, condensedphases, gases)

In [ ]:
specs = EquilibriumSpecs(system)
specs.pressure()
specs.enthalpy()

solver = EquilibriumSolver(specs)

In [ ]:
temperatures = []

table = Table()

for fAir in fAirs:
    state = ChemicalState(system)
    state.temperature(320.0, "°C")
    state.pressure(25.0, "bar")
    state.set("Fuel", 1.0, "mol")
    state.set("O2", fAir * factor * 0.21, "mol")
    state.set("N2", fAir * factor * 0.79, "mol")

    props = ChemicalProps(state)
    conditions = EquilibriumConditions(specs)

    conditions.pressure(props.pressure())
    conditions.enthalpy(heatDuty, "kJ")

    conditions.setLowerBoundTemperature(298.15, "°C")
    conditions.setUpperBoundTemperature(3000.0, "°C")

    result = solver.solve(state, conditions)

    assert result.succeeded()

    table.column("Air:Fuel") << fAir
    table.column("Temperature") << state.temperature()
    table.column(f"n(Gases)") << state.props().phaseProps("GaseousPhase").amount()

    if output_all_species:
        for species in system.species():
            table.column(f"n({species.name()})") << state.speciesAmount(species.name())
    else:
        table.column("n(C(gr))")    << state.speciesAmount("C(gr)")
        table.column("n(CH4)")      << state.speciesAmount("CH4")
        table.column("n(CO)")       << state.speciesAmount("CO")
        table.column("n(CO2)")      << state.speciesAmount("CO2")
        table.column("n(H2)")       << state.speciesAmount("H2")
        table.column("n(H2O)")      << state.speciesAmount("H2O")
        table.column("n(N2)")       << state.speciesAmount("N2")

In [ ]:
table.save("output.txt")

In [ ]:
print(table)

Air:Fuel | Temperature | n(Gases) |       n(e-) |        n(C) |       n(C+) |       n(C-) |       n(CH) |      n(CH+) |      n(CH2) |      n(CH3) |    n(CH2OH) |   n(CH2OH+) |     n(CH3O) |      n(CH4) |    n(CH3OH) |   n(CH3OOH) |       n(CN) |      n(CN+) |      n(CN-) |      n(CNN) |    n(CO) |      n(CO+) |   n(CO2) |     n(CO2+) |     n(COOH) |       n(C2) |      n(C2+) |      n(C2-) |      n(C2H) | n(C2H2,acetylene) | n(C2H2,vinylidene) | n(CH2CO,ketene) |  n(O(CH)2O) | n(HO(CO)2OH) | n(C2H3,vinyl) |    n(CH3CN) | n(CH3CO,acetyl) |     n(C2H4) | n(C2H4O,ethylen-o) | n(CH3CHO,ethanal) |  n(CH3COOH) | n(OHCH2COOH) |     n(C2H5) |     n(C2H6) | n(CH3N2CH3) |   n(C2H5OH) |  n(CH3OCH3) | n(CH3O2CH3) |      n(CCN) |      n(CNC) |     n(OCCN) |     n(C2N2) |      n(C2O) |       n(C3) | n(C3H3,1-propynl) | n(C3H3,2-propynl) | n(C3H4,allene) | n(C3H4,propyne) | n(C3H4,cyclo-) | n(C3H5,allyl) | n(C3H6,propylene) | n(C3H6,cyclo-) | n(C3H6O,propylox) | n(C3H6O,acetone) | n(C3H6O,propanal) | 

We now plot the calculated temperatures of the exit stream as a function of the fuel-to-air mass ratio.

In [ ]:
from reaktplot import *

fig = Figure()
fig.xaxisTitle("Air:Fuel (mass ratio)")
fig.yaxisTitle("Temperature [K]")
fig.drawLineWithMarkers(fAirs, table["Temperature"], name="")
fig.height(1000)
fig.show()